In [ ]:
import numpy as np
import pandas as pd


import re
from datetime import datetime, date, timedelta


import matplotlib.pyplot as plt
import seaborn as sns
from to_img import to_img



plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 17
import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")
sns.set(style="ticks", color_codes=True)

## 1. Data

The data consists of text scraped from every search result for 'data science/scientist' on Indeed.com using the [Requests](https://docs.python-requests.org/en/master/ "Requests Library"), Tor, and [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/ "BeautifulSoup") libraries.

### 1.1 Load and Inspect Data

In [ ]:
# note the change in memory usage.
data = pd.read_csv('../app/data/total.csv')
print(len(data))


In [ ]:
to_img(data.head(),name='1.1_loaded_data')
data.info(memory_usage='deep')
data.head()


##### Data Dictionary
company<br>
    - String, company names

description<br>
    - String, full text description of job

estimated_salary<br>
    - String, Indeed's salary estimation

extractDate<br>
    - String(?), date record was webscrapped

job_type_items<br>
    - String, formatted list of 'full-time', 'part-time', etc

location<br>
    - String, state and city/remote

postDate<br>
    - String, date job posting was created

rating<br>
    - Float, Indeed.com ascribed company ratings

raw_desc_soup<br>
    - String, raw full description, retained for possible future use/analysis

requirements<br>
    - String, employer-listed educational, aptitude, and experience requirements

sal_guide_items<br>
    - String, formatted list containing redundant characters and the Indeed salary estimate

salary<br>
    - String, if given by employer

salary_and_jType<br>
    - String, formatted list combining salary from salfromsection and job_type_items

salfromsection<br>
    - String, salary, if given, extracted from job description

summary<br>
    - String, brief summaries of each job extracted from sach results

title<br>
    - String, job position title

url<br>
    - String, job posting/description address

### 1.1 Load & Inspect Data

In [ ]:
data = pd.read_csv('../app/data/total.csv')
# 1.1.1 Inspect Table
data.info(memory_usage='deep')
data.head()

In [ ]:
print(data.salary[-10:])

In [ ]:
null = sum(data.salary.isnull())
nnull = sum(data.salary.notnull())
print('')
print(f'The data contains {data.shape[0]} rows of individual job postings with values for {data.shape[1]} columns based on\n different sections of each post\'s web page.')
print('')
print(f'- Out of {len(data)} job postings {nnull} or {round(nnull/len(data)*100,2)}% include some sort of salary information,\n- The remaining {null} rowws, or {round(null/len(data)*100,2)}% are missing salary data.\n- Also, there are no null values in the rest of the table meaning we won\'t need\n  to lose any data by dropping rows. While the null values in the Pay column will\n  simply be dropped after we translate the not nulls into the target.')

    The following table is a view of the raw data from the scrape. I'll build the target by reducing the pay column to floating point values; this will require text cleaning which will also be conducted for the rest of the table. Location will be split into new columns for City and State while and because Indeed's search results show dates relative to the date the query was made I'll need to use PostDate and ExtractDate build a DatePosted column. It is important to note that JobUrl and date-related columns will not be converted into features for the logistic regression, instead they will be reattached to the data at the end in order to provide additional information and insights through the EC2 application. The only rows that'l  be using for modelling are JobTitle, Company, Summary, Requirements, and Description.

In [ ]:
shape = data.shape
print(f"The {shape[0]} rows of data have {shape[1]} features. Since this is a classification problem I'll be selecting one to be the target, leaving me with {shape[1]-1} features to work with.")

In [ ]:
#check for dupes
print(len(data[data.duplicated()]))


In [ ]:
data[data.company == 'The Getch']

In [ ]:
data.sal_guide_items[-5:-4]

In [ ]:
#check for missing
print(data.isna().sum().sort_values(ascending=False))

The likelihood of seeing duplicates coming from the raw scrapping is very unlikely, but this is still a good check to do in case something changes with Indeed.com's front-end code.

Missing values are another story. It seems that Indeed.com doesn't require employers to provide values for every feature, this certainly seems to be the case with requirements, not to mention everywhere salary information coud be held.\
Others, like estimated salary and ratings are generated by Indeed.com but don't exist for every posting.\
job_type_items and salary_and_jType will likely be collapsed into a new column for job types, I'll also do the same with all of the salary related features to build the model's target.

In [ ]:
# 1.1.2 Inspect & Modify Columns
data.nunique().sort_values(ascending=False)


In [ ]:
data.columns = map(str.lower, data.columns)
# no need to change many column names because they're about to be dropped.
data = data.rename({'title': 'job_title'}, axis=1)

## 2. Preprocesing A
### 2.1 Reinspect Data

In [ ]:
data.salary.unique()

In [ ]:
data.info(memory_usage='deep')
data.head()

### 2.2 Diagnose Data
In order to proceed with any statistical or exploratory data analysis I will need to wrangle the data quite a bit.
Here, I'll list each of the current feature columns and briefly describe what I'll need to do with them.

In [ ]:
# converting to lower and removing ounctuation/special charcters
#TODO remember to remove punct/spec - NPL may take care of this.
data.company.unique()[:10]

'description'\
Each value is a unique string of the full job description. I'll lower and remove special characters but this feature will be dealt with seperately in its own NLP process.


In [ ]:
data.description[0]

'estimated_salary'\
Indeed.com provides its own salary estimates in what appear to be bins.
These and all other salary ranges will be collapsed to their median values once I've extracted and converted their numeric string reresentations.


In [ ]:
data.estimated_salary.unique()[:5]

'sal_guide_items'\
While I could convert these to lists, the only information I need (salary) can be parsed out.

In [ ]:
data.sal_guide_items.unique()[:5]

'salary_and_jtype' & 'salfromsection'\
These will be wrangled in much the same way. The only other pertinent information in these features would be the job_type data already found in job_type_items,

In [ ]:
data.salary_and_jtype.unique()[:3]

In [ ]:
data.salfromsection.unique()[:5]

'salary'\
Several different approaches will need to be used to extract salary information from this feature because employers provide this information themselves.
For one example, the pay periods of hour, day, week etc will need to be scaled to an anuual scale so I can have a standard scale to start wiith.
The salary ranges also appear here, so I'll be extracting numeric string characters, converting them into floats representing the lower and upper bounds of each range, and taking their medaian value.

In [ ]:
data.salary.unique()[:5]

'extractdate' & 'postdate'\
extract_date is a string showing the day the record was webscrapped.
post_date is a string description of the relative age of each post from the date it was posted to the day it was webscrapped.
Converting post_date to a number will give me the the posts ae, which I'll then subtract from it's extract date in order to build a new feature carrying values for the DATE each job posting has been submitted to Indeed.com.


In [ ]:
data.postdate.unique()[:5]

In [ ]:
data.extractdate.unique()[:5]

'job_type_items'\
    I'm going to treat these as likert values, converrting each of these string formatted lists into an ordinal values for a new column.


In [ ]:
data.job_type_items.unique()

'location'\
This feature holds a wealth of information: state, city/remote, and in some cases a zip code.
Zip codes seem to be a recent addition to the information I can get from Indeed.com but since its presence is rather sparse I'll just extract city/remote and state values, splitting them into their respective columns.

In [ ]:
data.location.unique()[:5]

'rating'\
As the only feature coming in with numeric (float) values I should just need to scale it. I'll address missing values by imputing zero. The rationale behind this is that an unrated company should simply be rated as zero.

In [ ]:
print(sorted(data.rating.unique()))

'raw_desc_soup'\
I'm only saving this in case I refine or change my parsing. Below is a truncated slice of one of the raw records.

In [ ]:
data.raw_desc_soup[1][:1000]

'requirements'\
I will most likely merge this in with the description prior to conducting NLP. The rationale is that these explicit requirements may be echoed in the description or summary, so by adding them to the corpus I'll raise the scores derived from their increaseed frequency. Eventually I wiuld like to build a set of requirements that I can use in with a clustering algorithm, like KNN, to find/define subgroups within the sample population.

In [ ]:
data.requirements[0]

'summary'\
Much like the description, these are unique. COming from the search results page itself, these are brief descriptions of each job. In most cases this is not merely an abridgement of the summary. Like 'requirement' I'll start by lumping these texts in with description so analysis of the corpus can be boosted by the added descriptions.

In [ ]:
for i in data.summary[:3]:
    print(i)

'job_title'\
Although the webscrapper searches for 'data scientist' the results contain over 80 unique job titles. This is largely due to a prefix or suffix that provides the role's specialization at that particular company for that particular job posting. These include items like 'Marketing', 'Senior', 'Geospatial...'.
I'll start by cleaning and simplifying these values, i.e. appearences of 'Sr' can be converted to 'Senior'. This will significantly reduce the number of unique values, hopefully increasing model accuracy.

In [ ]:
print(len(data.job_title.unique()))
data.job_title.unique()[:10]

'url'\
Like raw_doc_soup, I'm simply keeping these in case I need to go back and re-parse the raw data. In fact, this URL points directly to the page where that raw doc soup is found.
I may also be able to detect job post updates and duplicates by using these unique addresses.

In [ ]:
data.url[0]

### 2.3 Wrangling & Feature Engineering
Below are the implementations of the data wrangling steps I described above for each feature. Using Numpy, Regex, and Pandas I'll simutaneaously clean the strings, extract and convert numeric information, and create new feature columns.

In [ ]:

def sal_chars(data):
    """When applied, substitutes '' for any listed special characters, 

    Args:
        data (column values): I call this function on specific columns. Data, in this case, represents that column and its values.

    Returns:
        string: the original string values, sans any listed special characters.
    """
    cleaned = re.sub(r'[\n|,|+|$|[|$|\'|"]',r'',data)
    return cleaned

def sal_splitter(data):
    """Splits off salary information from a string formatted list

    Args:
        data (column values):  I call this function on specific columns. Data, in this case, represents that column and its values.

    Returns:
        string: returns the split containing salary information.
    """
    x = data.split(',')
    return x[2]

def Pay_period(data):
    """Used in the creation of a temporary column that will be discarded once I use it to generate a annualized salary column.

    Args:
        data (string): I call this function on specific columns. Data, in this case, represents that column and its values.

    Returns:
        string: pay rate descripions
    """
    z = ''
    substrings = ['hour','day','week','year']
    for i in substrings:
        if i in data:
            return i
        else:
            continue

def jobtype(data):
    """Used for creating a new feature containing nominal values for the job type.

    Args:
        data (string): Values from the 'job_type_items' column.

    Returns:
        string: Extracted and simplified values for each category.
    """
    if type(data) == str:
        if re.search('full', data):
            out = 'full'
        if re.search('part', data):
            out = 'part'
        if re.search('contract', data):
            out = 'contract'
        if re.search('temp', data):
            out = 'temp'
        return out
    else:
        pass

def annual(data):
    """Annualizes salary data

    Args:
        data (dataframe): the entire dataframe is passed through.

    Returns:
        string: In the new 'annual_sal' column, creates annualized salaries by multiplying them against the observed (and previously extracted) pay rates.
    """
    data['annual_sal'] = np.nan
    data['annual_sal'] = np.where(data['schedule'].str.contains("hour"), data.salary*40*(365/7), data['salary'])
    #data['annual_sal'] = np.where(data['schedule'].str.contains("day"), data['salary']*365/7*5, data['salary'])
    #data['annual_sal'] = np.where(data['schedule'].str.contains("week"), data['salary']*365/7, data['salary'])
    #data['annual_sal'] = np.where(data['schedule'].str.contains("month"), data['salary']*365/12, data['salary'])
    #data['annual_sal'] = np.where(data['schedule'].str.contains("year"), data['salary']/100, data['salary'])
    return data

def split_sal(i):
    try:
        lst = i.split('-',1)
        x = lst[0]
        y = lst[1]
        
        y = re.sub(r'[–]',r'',y)

        return (float(x)+float(y))/2
    except:
        return i

In [ ]:
salary = data[data.salary.notnull()]
data['salary'] = salary['salary'].apply(sal_chars)


In [ ]:


estimated_salary = data[data.estimated_salary.notnull()]
data['estimated_salary'] = estimated_salary['estimated_salary'].apply(sal_chars)


In [ ]:

salfromsection = data[data.salfromsection.notnull()]
data['salfromsection'] = salfromsection['salfromsection'].apply(sal_chars)

salary_and_jtype = data[data.salfromsection.notnull()]
data['salary_and_jtype'] = salary_and_jtype['salary_and_jtype'].apply(sal_chars)




In [ ]:


sal_guide_arr = data[data.sal_guide_items.notnull()]
data['sal_guide_items'] = sal_guide_arr['sal_guide_items'].apply(sal_splitter)

sal_guide_arr = data[data.sal_guide_items.notnull()]
data['sal_guide_items'] = sal_guide_arr['sal_guide_items'].apply(sal_chars)


In [ ]:


estimated_salary = data[data['estimated_salary'].notnull()]
data['Schedule1'] = estimated_salary['estimated_salary'].apply(Pay_period)

sal_guide_items = data[data['sal_guide_items'].notnull()]
data['Schedule2'] = sal_guide_items['sal_guide_items'].apply(Pay_period)

salary = data[data['salary'].notnull()]
data['Schedule3'] = salary['salary'].apply(Pay_period)

salary_and_jtype = data[data['salary_and_jtype'].notnull()]
data['Schedule4'] = salary_and_jtype['salary_and_jtype'].apply(Pay_period)

salfromsection = data[data['salfromsection'].notnull()]
data['Schedule5'] = salfromsection['salfromsection'].apply(Pay_period)

val_cols = ['Schedule1','Schedule2','Schedule3','Schedule4','Schedule5']
data['schedule'] = data[val_cols].bfill(axis=1).iloc[:, 0]
data.drop(val_cols,inplace=True,axis=1)



In [ ]:
def splitter(data):
    lst = []
    if type(data) ==  str:
        for val in re.findall('([0-9\.]+)',data):
            lst.append(val)
        if len(lst) > 1:
            return (float(lst[0])+float(lst[1]))/2
        #



            

                    
                
     
            
        
        


data['estimated_salary'] = data['estimated_salary'].apply(splitter)
data['salary'] = data['salary'].apply(splitter)
col = ['estimated_salary','salary']
data['salary'] = data[col].bfill(axis=1).iloc[:, 0]
data

In [ ]:


cols = ['company','description', 'job_type_items','location','postdate','requirements',	'summary','job_title']
for txt_col in cols:
    data[txt_col] = data[txt_col].str.lower()



data['jobtype'] = data.job_type_items.apply(jobtype)

data['text'] = data.description+' '+data.summary
#TODO add requirements

cols = ['job_type_items','raw_desc_soup','url','description','summary','requirements']


data.text = data.text.replace(')',' ')



data = annual(data)
#data.schedule.value_counts() keep and eye on this, may have to deal with more than just hourly

In [ ]:
def reducer(data):
    if data > 1000:
        return round(data/1000)
    elif data > 1:
        return round(data)
    else:
        return data
    
data['final_sal'] = data.annual_sal.apply(reducer)



def restorer(data):
    if data > 1:
        return data*1000
    else:
        return data
data['final_sal'] = data.final_sal.apply(restorer)

In [ ]:
data

Let's see what the data looks like at this point. It looks like the 17 original features have been reduced to 11. Most of the columns have been collapsed and cleaned, too. Just a bit more work till this is ready for EDA, NLP, and transformations.

In [ ]:
data.salary

In [ ]:
def postD(data):
    """Critical function that converts postdate values into relative (in days) distance from the extract date.

    Args:
        data (string): text descrption of how old the post is.

    Returns:
        npnan or int: numeric representation of post age
    """
    if data != np.nan:
        rid = ['active','%+ days ago','+']
        for i in rid:
            if i in data:
                return np.nan
        
        rid2 = ['just posted','today','postedjust posted','postedtoday','hiring ongoing']
        for i in rid2:
            if i in data:
                return 0   
            
        rid3 = ['posted 1 day ago']
        for i in rid3:
            if i in data:
                return 1
        
        rid3 = ['posted1 day ago']
        for i in rid3:
            if i in data:
                return 1


    #data['dateposted'] = data['dateposted'].astype('int')
    return data

def pDate(row):
    """Subtracts the distance (in days) provided by the postD function.
    Args:
        row (record in the data): record

    Returns:
        datetime object: extract date - relative age = actual post date
    """

    days_ago = int(row['dateposted'])
    delta = timedelta(days_ago)
    try:
        return row['extractdate'] - delta
    except:
        return row

def sal_fixer(data):
    """Ensures that all salaries are on the same annualized scale.

    Args:
        data (float): values from the salary column

    Returns:
        float: annualized salary
    """
    if type(data) != np.nan:
        data = round(data,0)

        if len(str(data)) < 5:
            data*=1000
    else:
        pass
    return data

In [ ]:
data.postdate.value_counts()

In [ ]:
data['dateposted'] = data.postdate.apply(postD)


In [ ]:
data['extractdate'] = pd.to_datetime(data['extractdate'])


In [ ]:
data['dateposted'] = data.apply( lambda row : pDate(row), axis = 1)

In [ ]:
data

In [ ]:


data.rating = data.rating.fillna(0)

cols = ['extractdate','postdate','schedule','jobtype','salary']
data.drop(cols,inplace=True,axis=1)
data = data.rename({'annual_sal': 'salary'}, axis=1)

data['salary'] = data['salary'].apply(sal_fixer)

In [ ]:
data.salary

In [ ]:
def states_(i):
    """Extracts state values from location

    Args:
        i (string): values from location

    Returns:
        string: extracted state value
    """
    if ',' in i:
        fix =  i.partition(',')[2].strip()
        if len(fix) >2:
            return fix[:2]
        else:
            return fix
    if ',' not in i:
        return i
    elif 'remote' in i:
        return 'remote'


def cities_(i):
    """Extracts city values from location

    Args:
        i (string): values from location

    Returns:
        string: extracted city value
    """
    if ',' in i:
        return i.partition(',')[0]
    if ',' not in i:
        return i
    elif 'remote' in i:
        return 'remote'

data['state'] = data.location.apply(states_)
data['city'] = data["location"].apply(cities_)
#data[data.state.isnull()] = 'remote'

In [ ]:
data.columns
cols = ['salary','estimated_salary', 'job_type_items','raw_desc_soup', 'sal_guide_items','salary_and_jtype', 'salfromsection']
data.drop(cols,inplace=True,axis=1)
data.rename(columns={'final_sal': 'salary'},inplace=True)

In [ ]:
data = data[data.description != 'NaN']

In [ ]:
len(data)

In [ ]:
data.description.value_counts()[1:]

### 2.4 Assessment

In [ ]:
data.info(memory_usage='deep')
data.head()

In [ ]:
shape = data.shape
print(f"The {shape[0]} rows of data now have {shape[1]} variables whose values are now cleaned and typed properly.\nThe only missing values are in Salary which is my target.\nNext, I'll conduct analysis, encoding, and scaling of each feature, starting with Salary which .")
data.describe(include = 'all')


In [ ]:
sns.countplot(x='salary', data=data.notnull(), palette='Set3')
plt.show()

In [ ]:
null = len(data[data.salary.isnull()])
nnull = len(data[data.salary.notnull()])
print(f'{round(nnull/len(data)*100,2)}% of the data has salary information, should be an easy target')

In [ ]:
data = data[data['text'].notna()]

In [ ]:
data.to_csv(f'../app/data/wrangled_data.csv', index=False)

In [ ]:
data

In [ ]:
munged = data
munged

In [ ]:
null = len(munged[munged.salary.isnull()])
nnull = len(munged[munged.salary.notnull()])
print(f'Droping {len(data)-len(munged)} duplicates and out of date posts leaves {len(munged)} rows of data for the regressor to be trained and tested on.')
print(f'- Of those {len(munged)} job postings {nnull} or {round(nnull/len(munged)*100,2)}% include salary information,\n- The remaining {null} rows, or {round(null/len(munged)*100,2)}% are missing salary data.')

## Detecting Outliers: Using both the Z-Score and IQR methods
    Now that preprocessed the data and building the target variable of Salary let's see conduct a bit more EDA to see if there are any outliers that could potentially skew how the regressor will learn from the training data.

In [ ]:
ax = sns.pairplot(data=munged[munged.salary.notnull()],
                  x_vars=['dateposted'],
                  y_vars=['salary'], height=8, hue="salary", palette="crest")

ax.fig.set_size_inches(18,4)

A plot is helpful in isualizing data, but let's program a few functions to detect and remove outliers based on thresholds.
With the Z-Score, we can determine any data outside 3 standard deviations from the mean of our salary data to be an outlier.
Wheras with the interquartile (IQR) range, the middle 50% of given salaries, I'll set the conventional cutoff of 1.5+/- the IQR as the cutoff.
Values found outside of either range will be collected into a list and that'll allow for some fancy indexing so those particular postings can be examined and removed programatically.

#### Z-Score
Salaries 3 standard deviations away from the mean will be listed as outliers.

In [ ]:
outliers = []
def z_detect(munged):
    sample = munged[munged['salary'].notnull()].salary
    threshold=3
    mean = np.mean(sample)
    std = np.std(sample)
    
    for i in sample:
        z_score = (i-mean)/std
        if np.abs(z_score) > threshold:
            outliers.append(i)
    if len(outliers) == 0:
        pass
    else:
        return outliers
z_detect(munged)

In [ ]:
munged

#### IQR
Salaries outside 1.5 times the interquartile range boundaries, either above or below will be listed as outliers.

In [ ]:
def iqr_detect(munged):
    sample = munged[munged['salary'].notnull()].salary
    Q1, Q3 = np.percentile(sample,[25,75])
    iqr = Q3-Q1
    lower_bound = Q1-(1.5*iqr)
    upper_bound = Q3+(1.5*iqr)
    for i in sample:
        if (i < lower_bound)  | (i > upper_bound):
            outliers.append(i)
    if len(outliers) == 0:
        pass
    else:
        return outliers
set(iqr_detect(munged))

In [ ]:
def unique(list1):
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    for x in unique_list:
        print(x)
        
mn = min(outliers)
mx = max(outliers)
print(f'The {len(set(outliers))} Unique Outliers Are:')
unique(outliers)
print(f'- With a minumum of ${round(mn)} and a maximum of ${round(mx)}')

In [ ]:
# Take a look at those tables.

outliers = munged[munged['salary'].isin(outliers)]
munged.drop(outliers.index, axis=0,inplace=True)
outliers

In [ ]:
null = len(munged[munged.salary.isnull()])

nnull = len(munged[munged.salary.notnull()])

In [ ]:
ax = sns.pairplot(data=munged[munged.salary.notnull()],
                  x_vars=['dateposted'],
                  y_vars=['salary'], height=8, hue="salary", palette="crest")

ax.fig.set_size_inches(18,4)

In [ ]:
sns.countplot(x='salary', data=munged.notnull(), palette='Set3')
print(f'- Dropping {len(outliers)} outliers now leaves {nnull}, or {round(nnull/len(munged)*100,2)}%, of rows with with salary information,\n- The remaining {null} rows, or {round(null/len(munged)*100,2)}%, are missing salary data.')


In [ ]:
munged.info()

In [ ]:
munged.to_csv(f'../app/data/wrangled_data.csv', index=False)

In [ ]:
munged.info()

In [ ]:
munged.info(memory_usage='deep')

In [ ]:
munged.drop('location',inplace=True,axis=1)

In [ ]:
munged.info(memory_usage='deep')